<a href="https://colab.research.google.com/github/signnd/proposal-ta/blob/main/k_fold_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from: https://github.com/SadmanSakib93/Stratified-k-fold-cross-validation-Image-classification-keras/blob/master/stratified_K_fold_CV.ipynb

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls
import sys
# This is the path to where in google drive the code is stored!
root_path = '/content/drive/My Drive/'
sys.path.append(root_path)

Mounted at /content/drive
drive  sample_data


In [ ]:
dataset_folder_name = os.path.join(root_path, 'all_new_data')
MODEL_FILENAME = root_path+"model_cv.h5"
source_files = []
class_labels = ['a', 'ba', 'ca', 'da', 'ga', 
                'ja', 'ka','la','ma', 'na', 
                'nga', 'nya','pa', 'ra', 'sa', 
                'ta', 'wa', 'ya']
X = []
Y = []

img_rows, img_cols = 100, 100 # input image dimensions
train_path = os.path.join(dataset_folder_name, 'train')
validation_path = os.path.join(dataset_folder_name, 'validation')
test_path = os.path.join(dataset_folder_name, 'test')

In [ ]:
def transfer_between_folders(source, dest, split_rate):
    """ Based on the split ratio this function moves some portion of the source folder to destination folder!

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest locaiton

    """
    global source_files
    source_files = os.listdir(source)
    if(len(source_files) != 0):
        transfer_file_numbers = int(len(source_files)*split_rate)
        transfer_index = random.sample(
            range(0, len(source_files)), transfer_file_numbers)
        for each_index in transfer_index:
            shutil.move(os.path.join(source, str(source_files[each_index])), os.path.join(
                dest, str(source_files[each_index])))

    else:
        print("No file moved. Source empty!")


def transfer_all_class_between_folders(source, dest, split_rate):
    """ Transfer the files from source to dest for all the classes. This function calls the 'transfer_between_folders' to actually perform the transfer.

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest locaiton

    """
    for label in class_labels:
        transfer_between_folders(os.path.join(dataset_folder_name, source, label),
                                 os.path.join(
                                     dataset_folder_name, dest, label),
                                 split_rate)


def my_metrics(y_true, y_pred):
    """ Calculate accuracy, precision, and f1 score of the model's prediction with respect to true labels.

        Args:
            y_true: list/array
                All true class labels
            y_pred: list/array
                All predicted class labels

        Returns:
            accuracy: float
                Accuracy measure of the model
            precision: float
                Precision measure of the model
            f1_Score: float
                F1-score measure of the model

    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    f1_Score = f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1_Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1_Score

In [ ]:
transfer_all_class_between_folders('test','train', 1.0)

In [ ]:
transfer_all_class_between_folders('train', 'test', 0.20)

In [ ]:
def prepare_name_with_labels(folder_name, dataset_type='train'):
    """ Prepare the file names (X) and the class labels (Y) from folder location of images.

        Args:
            folder_name: str
                Source folder's path

    """
    source_files = os.listdir(os.path.join(dataset_folder_name, dataset_type, folder_name))
    y_label = 0
    for i in range(len(class_labels)):
        if(folder_name == class_labels[i]):
            y_label = i
    for val in source_files:
        X.append(val)
        Y.append(y_label)

In [ ]:
# Organize file names and class labels in X and Y variables
for i in range(len(class_labels)):
    prepare_name_with_labels(class_labels[i])

In [ ]:
X = np.asarray(X)
Y = np.asarray(Y)

In [ ]:
batch_size = 512
epoch = 100
num_of_channels = 3
number_of_class_labels = len(class_labels)

In [ ]:
"""
    Note that, this model structure is a very basic one. 
    To achieve better performance, you should change the model structure and hyperparameters according to your needs and data. 
    So, optimize the structure of the model!
"""

def get_model():
    activation_function = 'relu'

    model = Sequential()
    model.add(Conv2D(64, (3, 3), padding='same',
                     activation=activation_function, input_shape=(img_rows, img_cols, num_of_channels)))
    model.add(Conv2D(64, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(32, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(16, (3, 3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(16, (3, 3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(64, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation=activation_function))
    model.add(Dropout(0.1))
    model.add(Dense(number_of_class_labels, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


model = get_model()

In [ ]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
fold_num = 0
for train_index, val_index in skf.split(X, Y):
    # First cut all images from validation to train (if any exists)
    transfer_all_class_between_folders('validation', 'train', 1.0)
    fold_num += 1
    print("Results for fold", fold_num)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for each_index in range(len(X_val)):
        class_label = ''
        for i in range(len(class_labels)):
            if(Y_val[each_index] == i):
                class_label = class_labels[i]
        # Then, copy the validation images to the validation folder
        shutil.move(os.path.join(dataset_folder_name, 'train', class_label, X_val[each_index]),
                    os.path.join(dataset_folder_name, 'validation', class_label, X_val[each_index]))

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.20,
        fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)
    
    # fit model
    history = model.fit(train_generator,
                        epochs=epoch)

    predictions = model.predict(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    
    # evaluate validation performance
    print("***Performance on Validation data***")
    val_acc, val_prec, val_fScore = my_metrics(true_classes, y_predictions)

Results for fold 1
Found 1098 images belonging to 18 classes.
Found 199 images belonging to 18 classes.
Epoch 1/100
3/3 [==============================] - 364s 172s/step - loss: 2.8895 - accuracy: 0.0574
Epoch 2/100
3/3 [==============================] - 6s 2s/step - loss: 2.8883 - accuracy: 0.0692
Epoch 3/100
3/3 [==============================] - 5s 2s/step - loss: 2.8854 - accuracy: 0.0710
Epoch 4/100
3/3 [==============================] - 5s 1s/step - loss: 2.8759 - accuracy: 0.0692
Epoch 5/100
3/3 [==============================] - 5s 1s/step - loss: 2.8559 - accuracy: 0.1011
Epoch 6/100
3/3 [==============================] - 5s 1s/step - loss: 2.8306 - accuracy: 0.0956
Epoch 7/100
3/3 [==============================] - 6s 2s/step - loss: 2.7888 - accuracy: 0.1157
Epoch 8/100
3/3 [==============================] - 5s 1s/step - loss: 2.7131 - accuracy: 0.1302
Epoch 9/100
3/3 [==============================] - 6s 3s/step - loss: 2.6531 - accuracy: 0.1439
Epoch 10/100
3/3 [==========

In [ ]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False 
)
predictions = model.predict(test_generator, verbose=1)
y_predictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

test_acc, test_prec, test_fScore = my_metrics(true_classes, y_predictions)
#model.save(MODEL_FILENAME)

==============TEST RESULTS============
Found 234 images belonging to 18 classes.
1/1 [==============================] - 2s 2s/step
Accuracy  : 0.9358974358974359
Precision : 0.9397093647093647
f1Score : 0.93551530773753
[[13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 11  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 11  0  0  0  0  0  0  0  0  0  2  0  0  0  0]
 [ 0  0  0  0 10  0  0  1  0  0  0  0  0  0  0  2  0  0]
 [ 0  0  1  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0 12  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 13  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 13  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  1 11  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 12  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  